In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

import sys
sys.path.append("../")
import src.soporte as sp

In [3]:
df = pd.read_csv("../data/train.csv")

In [4]:
df.drop(["id"], axis = 1, inplace = True)

In [5]:
sp.analisis_basico(df)

+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+

Estructura de los datos: (40455, 10)


,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.30,Premium,D,SI2,62.4,58.0,4.31,4.28,2.68,6.353
1,1.01,Ideal,E,VVS2,62.7,56.0,6.42,6.46,4.04,9.183


+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+

Número de filas duplicadas:
85
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+

Columnas, Nulos y Dtypes:


,nulos,dtypes
carat,0,float64
cut,0,object
color,0,object
clarity,0,object
depth,0,float64
table,0,float64
x,0,float64
y,0,float64
z,0,float64
price,0,float64


+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+

Descripción de las variables tipo Numéricas:


,count,mean,std,min,25%,50%,75%,max
carat,40455.0,0.797576,0.475939,0.200,0.400,0.70,1.040,5.010
depth,40455.0,61.746377,1.436796,43.000,61.000,61.80,62.500,79.000
table,40455.0,57.451694,2.234569,44.000,56.000,57.00,59.000,95.000
x,40455.0,5.729163,1.125244,0.000,4.710,5.69,6.540,10.740
y,40455.0,5.732896,1.154799,0.000,4.720,5.70,6.540,58.900
z,40455.0,3.537351,0.711002,0.000,2.910,3.52,4.030,31.800
price,40455.0,7.783822,1.016828,5.787,6.851,7.78,8.581,9.842


+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+

Descripción de las variables tipo Categóricas:


,count,unique,top,freq
cut,40455,5,Ideal,16230
color,40455,7,G,8410
clarity,40455,8,SI1,9779


+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+



In [ ]:
sp.distribucion_numericas(df)

In [ ]:
sp.correla_respuesta_num(df,"price")

In [ ]:
sp.correla_map(df)

In [ ]:
df.drop(["depth", "table"], axis = 1, inplace = True)

In [ ]:
sp.correla_respuesta_cate(df, "price")

In [ ]:
sp.outlier_boxplot(df, "price")

In [ ]:
especial = {"carat" : {"top" : 2.9},
            "x" : {"top" : 15},
            "y" : {"top" : 20},
            "z" : {"top" : 10}}
out = sp.detectar_outliers(df, "price", especial)


In [ ]:
df_sin_out = sp.tratar_outliers(df, out, "null")

In [ ]:
sp.outlier_boxplot(df_sin_out, "price")

In [9]:
def tratamiento_nulos_num(dataframe, metodo, valor = 0 , respuesta = None, neighbors = 5):
    columnas_numeric = dataframe.select_dtypes(include = np.number).columns
    if respuesta != None:
        columnas_numeric = columnas_numeric.drop(respuesta)

    if metodo == "drop":
        dataframe[columnas_numeric].dropna(how = "any", inplace = True)
        return dataframe
    
    elif metodo in ["replace", "mean", "median", "mode"]:
        if metodo == "replace":
            numericas_trans = dataframe[columnas_numeric].fillna(valor)
        else:
            for col in columnas_numeric:
                if metodo == "mean":
                    dataframe[col].fillna(df[col].mean()[0], inplace = True)
                elif metodo == "median":
                    dataframe[col].fillna(df[col].median()[0], inplace = True)
                elif metodo == "mode":
                    dataframe[col].fillna(df[col].mode()[0], inplace = True)
            return dataframe 
        
    elif metodo in ["iterative", "knn"]:
        if metodo == "iterative":
            imputer = IterativeImputer()
        elif metodo == "knn":
            imputer = KNNImputer(neighbors)

        numericas_trans = pd.DataFrame(imputer.fit_transform(df[columnas_numeric]), columns = columnas_numeric)
    dataframe.drop(columnas_numeric, axis = 1, inplace = True)
    dataframe[columnas_numeric] = numericas_trans

    return dataframe

In [ ]:
def tratamiento_nulos_cat(dataframe, metodo = "drop", valor = "desconocido", respuesta = None):
    columnas_object = dataframe.select_dtypes(include = "object").columns
    if respuesta != None:
        columnas_object = columnas_object.drop(respuesta)
        
    if metodo == "drop":
        dataframe[columnas_object].dropna(how = "any", inplace = True)

    elif metodo == "replace":
        categoricas_trans = dataframe[columnas_object].fillna(valor)
        dataframe.drop(columnas_object, axis = 1, inplace = True)
        dataframe[columnas_object] = categoricas_trans

    return dataframe

In [10]:
df["cut"].mode()

0    Ideal
Name: cut, dtype: object

In [12]:
columnas_object = df.select_dtypes(include = "object").columns